### Our aim is to build a predictive model for the churn rate in a bank
Let's get started!

In [ ]:
# import important libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import os

In [ ]:
# import our dataset for that bank for our research
directory_src = r'~/Desktop/AI-ML Requirments/Professional Track_ITIDA_Scholarship/Project_DSC_churn_prediction'
directory_dist = 'datasets/churnd'
file = 'churn.csv'

path = os.path.join(directory_src, file)
path_dist = os.path.join(directory_dist, file)
def load_data(path=path):
    # if the directory of our dataset doesn't exist, please create it.
    if not os.path.isdir(directory_dist):
       os.makedirs(directory_dist) 
    df = pd.read_csv(path)
    df.to_csv(path_dist)
    return df

In [ ]:
df = load_data() # loading and saving are done!

In [ ]:
# data investigations for gaining insights
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
# let's investigate the data numerical attributes
df.describe() # ~20.37% of the customers in our data exited.

In [ ]:
df[df.Exited==1] # 20.37% = 2037 rows

### let's do some visualizations including: histograms, scatterplots for gaining insights.

In [ ]:
df.hist(bins=25, figsize=(20, 15))
plt.show()

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(df[['Age', 'Balance',  'CreditScore', 'EstimatedSalary', 'Exited']], figsize=(15, 15))
plt.show()

### In my opinion, we still not able to grasp the insights and correlations with `Exited` attribute so let's try another technique.

In [ ]:
# let's split aour dataframe into numerical and categorical attributes for going in investigations in depth
df_num = df[list(df.describe())]
df_cat = df[['Surname', 'Geography', 'Gender']]

In [ ]:
df_num.head()

In [ ]:
df_cat.head()

In [ ]:
# let's take a copy of our numerical attributes dataframe for some playing with it/feature engineering processes for investigations.
df_num_ = df_num.copy()

In [ ]:
# let's visualize our linear correlation matrix for gaining insights
corr_mat = df_num_.corr()
sns.heatmap(corr_mat, linewidths=1.5)
corr_mat['Exited'].sort_values(ascending=False)

In [ ]:
# we will focus on the Age, Balance attributes for gaining insights

### We replace the `Age` attribute with the `AgeBucket` instead to help our model detect the impact of each Age stratum on the churn rate

In [ ]:
# we can see that the rate of churn for each startum of the age: has a very different impact on the churn rate.
df_num_['AgeBucket'] = df_num_.Age//20*(20)
print(df_num_[['AgeBucket', 'Exited']].groupby(['AgeBucket']).mean())
df_num_[['AgeBucket', 'Exited']].groupby(['AgeBucket']).mean().plot(kind='bar')
plt.grid()
plt.title('churn rate per age strata')
plt.ylabel('churn rate')
plt.xlabel('AgeBucket strata')
plt.legend(['Churn rate per age stratum'])
plt.show()

### We replace the `Balance` attribute with the `BalanceBucket` instead to help our model detect the impact of each Balance stratum on the churn rate¶

In [ ]:
# we can see that the rate of churn for each startum of the balance: has a very different impact on the churn rate.
df_num_['BalanceBucket'] = df_num_.Balance//50000*(50000)
print(df_num_[['BalanceBucket', 'Exited']].groupby(['BalanceBucket']).mean())
df_num_[['BalanceBucket', 'Exited']].groupby(['BalanceBucket']).mean().plot(kind='bar')
plt.grid()
plt.title('churn rate per balance strata')
plt.ylabel('churn rate')
plt.xlabel('BalanceBucket strata')
plt.legend(['Churn rate per balance stratum'])
plt.show()

### We replace the `CreditScore` attribute with the `CrScoreBucket` instead to help our model detect the impact of each Credit Card stratum on the churn rate¶

In [ ]:
df_num_.CreditScore.hist()

In [ ]:
# we can see that the rate of churn for each startum of the credit score: has a very different impact on the churn rate.
df_num_['CrScoreBucket'] = df_num_.CreditScore//125*(125)
print(df_num_[['CrScoreBucket', 'Exited']].groupby(['CrScoreBucket']).mean())
df_num_[['CrScoreBucket', 'Exited']].groupby(['CrScoreBucket']).mean().plot(kind='bar')
plt.grid()
plt.title('churn rate per credit score strata')
plt.ylabel('churn rate')
plt.xlabel('Credit scor strata')
plt.legend(['Churn rate per Credit score stratum'])
plt.show()

### All customers in the bank with a credit card score `less than 406`: 23/2037 customers, Exited!

In [ ]:
df[df.CreditScore<406]

In [ ]:
df[df.CreditScore<405].Exited.value_counts().index.values, df[df.CreditScore<406].Exited.value_counts().index.values

### We replace the `EstimatedSalary` attribute with the `SalaryBucket` instead to help our model detect the impact of each Salary stratum on the churn rate¶

In [ ]:
df.EstimatedSalary.hist()

In [ ]:
df_num_['SalaryBucket'] = df_num_.EstimatedSalary//35000*(35000)
print(df_num_[['SalaryBucket', 'Exited']].groupby(['SalaryBucket']).mean())
df_num_[['SalaryBucket', 'Exited']].groupby(['SalaryBucket']).mean().plot(kind='bar')
plt.grid()
plt.title('churn rate per salary strata')
plt.ylabel('churn rate')
plt.xlabel('salary strata')
plt.legend(['Churn rate per salary stratum'])
plt.show()

In [ ]:
df[df.EstimatedSalary<250000]['Exited'].value_counts()

#### With respect to the numerical continous attributes, we deduced that we should replace them with their corresponding categorical versions to help our model detect the pattern in our data very well

#### With respect to the numerical categorical attributes, they will be with no changes as they are well prepared with no missing values

### let's try combining some attributes to check their impact on the target attribute `Exited`

In [ ]:
# Balance/Tenure
df_num_['BalancePerTenure'] = df_num_.Balance/df_num_.Tenure

In [ ]:
df_num_['TenurePerAge'] = df_num_.Tenure/df_.Age

In [ ]:
df_num_[['BalancePerTenure', 'Exited']].groupby(['BalancePerTenure']).mean()

### Great Job! We could detect a strong variability in the `Exited` i.e. churn rate for each strata of the Balance Per Tenure attribute

In [ ]:
print(df_num_[['BalPerTenBucket', 'Exited']].groupby(['BalPerTenBucket']).mean())
df_num_[['BalPerTenBucket', 'Exited']].groupby(['BalPerTenBucket']).mean().plot(kind='bar')
plt.grid()
plt.title('churn rate per Balance per tenure strata')
plt.ylabel('churn rate')
plt.xlabel('Balance per tenure strata')
plt.legend(['Churn rate per Balance per tenure stratum'])
plt.show()

In [ ]:
# EstimatedSalary/Tenure
df_num_['SalaryPerTenure'] = df_num_.EstimatedSalary/df_num_.Tenure
df_num_['SalPerTenBucket'] = df_num_.SalaryPerTenure//35450*(35450)
print(df_num_[['SalPerTenBucket', 'Exited']].groupby(['SalPerTenBucket']).mean())
df_num_[['SalPerTenBucket', 'Exited']].groupby(['SalPerTenBucket']).mean().plot(kind='bar')
plt.grid()
plt.title('churn rate per Salary per tenure strata')
plt.ylabel('churn rate')
plt.xlabel('Salary per tenure strata')
plt.legend(['Churn rate per Salary per tenure stratum'])
plt.show()

In [ ]:
# let's try the balance/salary attribute
df_num_['BalancePerSalary'] = df_num_.Balance/df_num_.EstimatedSalary

In [ ]:
df_num_.head()

### let's try the combination of the credit/numOfproduts attribute

In [ ]:
df_num_['CreditPerProducts'] = df_num_.CreditScore/df_num_.NumOfProducts

In [ ]:
list(df_num)

In [ ]:
list(df_cat)

In [ ]:
df_cat.Geography.value_counts()

In [ ]:
df_cat.Gender.value_counts()

In [ ]:
df[df.Exited==1]['Surname'].value_counts()[:10]

### As long as that the `Age` attribute has the highest impact on the `Exited` (target) attribute; I am going to split the dataset into train and test based on this attribute so that all stratas of this attrbute to be represented in the test set as the train set for a better evaluation of the model

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
for train_idx, test_idx in splitter.split(df, df_num_.AgeBucket):
    train_df = df.loc[train_idx]
    test_df = df.loc[test_idx]
train_df.shape, test_df.shape   

I need to evaluate that our test dataset is including the same distribution of the values within the `AgeBucket` attribute as the same as the training dataset so as not to have a `sampling bias`in the model during evaluation

I am going to visualize the histogram for both the AgeBucket in both the train, test datasets

In [ ]:
# create the AgeBucket attribute in both the train, test datasets
train_df['AgeBucket'] = train_df.Age//15*(15)
test_df['AgeBucket'] = test_df.Age//15*(15)

In [ ]:
train_df.AgeBucket.hist()

In [ ]:
test_df.AgeBucket.hist() # identical histograms

### It's the time to build our transformers to preprocess our training dataset for the machine learning algorithm as we mentioned in the above cell

In [ ]:
df_ = train_df.copy()

### The basic steps for preprocessing implemented using a pipeline for automation:
**the following is with respect to numerical (continous and discrete) attributes.**
1. replace `Age` with `AgeBucket`
2. replace `Balance` with `BalanceBucket`
3. replace `CreditScore` with `CrScoreBucket`
4. create `TenurePerAge` attribute
5. create `BalancePerSalary` attribute
6. create `CreditPerProducts` attribute
7. drop `RowNumber`, `CustomerId`  attributes

**the following is with respect to categorical attributes.**
1. create `ExitedNameRatio` attribute instead of the `Surname` attribute
2. encode `Gender`, `Geography` using the OneHotEncoder

In [ ]:
df_.Tenure/df_.Age

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
idx_crscore, idx_age, idx_tenure, idx_balance, idx_products, idx_estsalary = 0, 1, 2, 3, 4, 7

class create_buckets(BaseEstimator, TransformerMixin):
      def fit(self, X, y=None):
          return self
      def transform(self, X, y=None):
          # convert the dataframe into 2d numpy array  
          data_array = X.values 
            
          # we aim to replace the Age attribute with it's bucket array  
          ageBucket = data_array[:, idx_age]//15*(15)
          balanceBucket = data_array[:, idx_balance]//50000*(50000)
          crscoreBucket = data_array[:, idx_crscore]//125*(125)
          tenureperage =  data_array[:, idx_tenure]/data_array[:, idx_age]
          balpersal = data_array[:, idx_balance]/data_array[:, idx_estsalary]  
          creditperproduct = data_array[:, idx_crscore]/data_array[:, idx_products]
            
          # replace now and add the new created attributes 
          data_array[:, idx_age] = ageBucket
          data_array[:, idx_balance] = balanceBucket
          data_array[:, idx_crscore] = crscoreBucket
          return np.c_[data_array[:, :-1], tenureperage, balpersal, creditperproduct]

In [ ]:
obj = create_buckets()
z = obj.fit_transform(df_)

In [ ]:
z[10:20]

In [ ]:
z.shape #Done!

In [ ]:
'Akubundu' in df['Surname'].values

In [ ]:
df_.Surname

In [ ]:
idx_surname = 0

def get_ratio(series_surname, series_exited, series_not_exited):
    ratio_exited = []
    # for each name in the surnames of our whole data
    for name_ in series_surname.values:
        val_exited = 0
        val_not_exited = 0
        total = 0
        if name_ in series_exited.index:
           val_exited = series_exited[name_]
        if name_ in series_not_exited.index:
           val_not_exited = series_not_exited[name_]
        total = val_exited + val_not_exited
        ratio_exited.append(val_exited/total)  
    return ratio_exited

class count_exited(BaseEstimator, TransformerMixin):
      def fit(self, X, y=None):
          # we aim to get the count of names in both cases when that name exited and didn't exit to get the ratio of exited with respect to didn't exit for that name 
          series_exited = X[X['Exited']==1]['Surname'].value_counts()
          series_not_exited = X[X['Exited']==0]['Surname'].value_counts()
          # we aim to get the list of counts for each exited name by get_freq(self.series_) function
          self.ratio_exited = get_ratio(X.Surname, series_exited, series_not_exited)  
          return self
      def transform(self, X, y=None):
          data_ = X.values
          ratios_ = np.array(self.ratio_exited)
          # we replace the surname attribute with the ratio of exited for that surname and then to drop the target attribute
          data_[:, 0] = ratios_
        
          return data_

In [ ]:
list(df_)

In [ ]:
obj = count_exited()
z = obj.fit_transform(df_)

In [ ]:
z.shape

In [ ]:
df_.Surname.value_counts()['Andrews']

In [ ]:
temp = df_[df.Exited==1]

In [ ]:
temp.Surname.value_counts()['Fanucci']

In [ ]:
temp2 = df_[df_.Exited==0]

In [ ]:
temp2.Surname.value_counts()['Fanucci']

In [ ]:
z[21:30]

In [ ]:
df_[df_.Exited==0]['Surname'].value_counts()[20:]

In [ ]:
x = np.array([90, 80, 56, 78, 90, 80, 56, 78, 90, 80, 56, 78]).reshape(3, 4)

In [ ]:
x

In [ ]:
x[:, 2:]

### Combine the output of the numerical and categorical attributes
Using the column transformer library.

idx_crscore, idx_age, idx_tenure, idx_balance, idx_products, idx_estsalary = 0, 1, 2, 3, 4, 7
idx_surname = 0

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

attr_list = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
full_pipe = ColumnTransformer([
    ('count_excited', count_exited(), ['Surname', 'Exited']),
    ('num_pipe', create_buckets(), attr_list),
    ('scaler', StandardScaler(), ['EstimatedSalary']),
    ('encode_geo_gender', OneHotEncoder(), ['Geography', 'Gender'])
])

In [ ]:
df_ = train_df.copy()

In [ ]:
# extracting the training labels
y_train = df_.Exited.copy()

In [ ]:
final_data = full_pipe.fit_transform(df_)

### Great Job ! We could transform our training dataframe `sucessfully` ;).

In [ ]:
final_data.shape

In [ ]:
final_data[8000:8020] #1, 7

In [ ]:
df_.iloc[8999]

### Great Transformation Job is done here... !

#### We aim to create our transformed dataframe on the training dataset and to save it for furture researches

In [ ]:
list_transformed_attrs = ['SurnamePercent', 'Exited', 'CreditCardBucket', 'AgeBucket', 'Tenure', 'BalanceBucket', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'TenurePerAge', 'BalancePerSalary', 'CreditPerProduct', 'EstSalaryScaled', 'Geo.France', 'Geo.Germany', 'Geo.Spain', 'Gen.Female', 'Gen.Male']

len(list_transformed_attrs)

In [ ]:
train_df_transformed = pd.DataFrame(final_data, columns=list_transformed_attrs)

In [ ]:
# let's see it's structure!
train_df_transformed.head()

In [ ]:
train_df_transformed.tail()

**Great!**

In [ ]:
# save this dataframe
train_df_transformed.to_csv('datasets/churnd/churn_train_transformed.csv', index=False)

### Let's start to train our model
I am going to try/fine-tune the following models for our Churn Rate Problem: `Binary Classification 1: Churned, 0: Didn't Churn`
1. `Stochastic Gradient Descent Classifier`
2. `Support Vector Machine Classifier`
3. `Desicion Tree Classifier`
4. `Random Forest Classifier`

Let's Get Started!

In [ ]:
# We aim here to remove the target labels away from our final training 2d-numpy array for training.
X_train = np.c_[final_data[:, 0], final_data[:, 2:]]

In [ ]:
X_train.shape, final_data.shape

In [ ]:
from sklearn.linear_model import SGDClassifier

model_SGD = SGDClassifier()

In [ ]:
model_SGD.fit(X_train, y_train)

In [ ]:
# evaluate model prediction on the training data
SGD_train_pred = model_SGD.predict(X_train)

In [ ]:
y_train[:10].values

In [ ]:
SGD_train_pred[:10]

**It looks that the stochastic gradient descent classifier is `underfitting` the training dataset with overall accuracy = 63.62%**
We will need to use a more powerful model for a better prediction.

In [ ]:
# let's evaluate the overall model accuracy on the training dataset
from sklearn.metrics import accuracy_score

train_accuracy = accuracy_score(y_train, SGD_train_pred)
train_accuracy*100

In [ ]:
# let's evaluate the model accuracy on multiple training rounds
from sklearn.model_selection import cross_val_score

cross_SGD_accuracy = cross_val_score(model_SGD, X_train, y_train, cv=10, scoring='accuracy')
(cross_SGD_accuracy.mean())*100

In [ ]:
# let's investigate the model confusion matrix on training dataset
from sklearn.metrics import confusion_matrix
conf_SGD = confusion_matrix(y_train, SGD_train_pred)
conf_SGD

In [ ]:
from sklearn.model_selection import cross_val_predict

cross_SGD_pred = cross_val_predict(model_SGD, X_train, y_train, cv=3)

In [ ]:
# let's investigate the model confusion matrix on multiple rounds of training
from sklearn.metrics import confusion_matrix
conf_SGD = confusion_matrix(y_train, cross_SGD_pred)
conf_SGD

In [ ]:
# let's see the model precision/recall/f1 metrics
from sklearn.metrics import precision_score, recall_score, f1_score

precision_score(y_train, cross_SGD_pred), recall_score(y_train, cross_SGD_pred), f1_score(y_train, cross_SGD_pred)

In [ ]:
# optimal classifier is:
conf_SGD_ = confusion_matrix(y_train, y_train)
conf_SGD_

In [ ]:
# let's investigate the SGradientDescent classifier Precision/Recall plot, ROC curve
from sklearn.metrics import precision_recall_curve

cross_SGD_score = cross_val_predict(model_SGD, X_train, y_train, cv=3, method='decision_function')
prec_sgd, recall_sgd, thre_sgd = precision_recall_curve(y_train, cross_SGD_score)

In [ ]:
def precision_recall_curve_plot(precisions, recalls, thresholds):
    # plot() is used for continous values
    plt.plot(thresholds, precisions[:-1], 'r--', label='Precision')
    plt.plot(thresholds, recalls[:-1], 'b-', label='recall')
    plt.xlabel('Threshold')
    plt.legend()
    plt.grid()
    plt.title('Precision - Recall Curve vs Threshold')
    plt.figure(figsize=(25, 10))
    plt.show()

In [ ]:
precision_recall_curve_plot(prec_sgd, recall_sgd, thre_sgd)

In [ ]:
def precision_recall_direct_plot(recalls, precisions):
    plt.plot(recalls[:-1], precisions[:-1])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.grid()
    plt.legend(['Recall-Precision'])
    plt.title('Precisin - Recall Curve Directly')
    plt.show()

In [ ]:
precision_recall_direct_plot(prec_sgd, recall_sgd)

In [ ]:
# let's investigate the model roc curve
from sklearn.metrics import roc_curve
fpr_sgd, tpr_sgd, thre_sgd_roc = roc_curve(y_train, cross_SGD_score)

In [ ]:
def roc_curve_plot(FPR, TPR):
    plt.plot(FPR, TPR, 'r-', label='FPR : TPR')
    plt.plot([0, 1], [0, 1], 'b--', label='diagonal') # plotting the diagonal of the ROC curve.
#     plt.plot([0, 0.4], [0.4, 0.98], 'b--', label='checked:)') # plotting the diagonal of the ROC curve.
    plt.legend()
    plt.grid()
    plt.xlabel('False Postive Rate')
    plt.ylabel('True Postive Rate')
    plt.title('ROC curve of TPRate, FPRate')
    plt.show()

In [ ]:
roc_curve_plot(fpr_sgd, tpr_sgd)

the stochastic gradient descent classifier is very poor on the training dataset and is considered as a very weak model

In [ ]:
# area under the roc curve
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train, cross_SGD_score)

In [ ]:
# let's try to train our support vector machine model
from sklearn.svm import SVC

model_SVC = SVC()

In [ ]:
model_SVC.fit(X_train, y_train)

Evaluate the SVC model on the training dataset

In [ ]:
SVC_train_pred = model_SVC.predict(X_train)
SGD_train_pred[:10]

In [ ]:
y_train[:10].values

In [ ]:
# evaluate the model accuracy on training dataset
accuracy_score(y_train, SVC_train_pred)*100

In [ ]:
# evaluate the confusion matrx on trainig dataset
confusion_matrix(y_train, SVC_train_pred)

### Much better results

In [ ]:
# evaluate on multiple rounds of trainig
cross_SVC_score = cross_val_score(SVC(), X_train, y_train, cv=10, scoring='accuracy')
(cross_SVC_score.mean())*100

In [ ]:
# evaluate the accuracy on multiple trainig rounds
cross_SVC_pred = cross_val_predict(SVC(), X_train, y_train, cv=10)
accuracy_score(y_train, cross_SVC_pred)*100

In [ ]:
confusion_matrix(y_train, cross_SVC_pred)

In [ ]:
cross_SVC_score = cross_val_predict(SVC(), X_train, y_train, cv=10, method='decision_function')
prec_svc, recall_svc, thre_svc = precision_recall_curve(y_train, cross_SVC_score)
precision_recall_curve_plot(prec_svc, recall_svc, thre_svc)
precision_recall_direct_plot(prec_svc, recall_svc)

**We got an overall poor performance as well with the support vector machine classifier**

In [ ]:
# evaluate the roc curve
fpr_svc, tpr_svc, thre_svc_roc = roc_curve(y_train, cross_SVC_score)
roc_curve_plot(fpr_svc, tpr_svc)
roc_auc_score(y_train, cross_SVC_score)

In [ ]:
# let's evaluate the decision tree classifier
from sklearn.tree import DecisionTreeClassifier

model_DTC = DecisionTreeClassifier()

In [ ]:
model_DTC.fit(X_train, y_train)

In [ ]:
DTC_train_pred = model_DTC.predict(X_train)

In [ ]:
y_train.values[:10]

In [ ]:
DTC_train_pred[:10]

### For sure, our decision tree classifier reached accuracy = 100% on training dataset indicating **Overfitting** problem

In [ ]:
accuracy_score(y_train, DTC_train_pred)*100

In [ ]:
cross_DTC_train = cross_val_score(model_DTC, X_train, y_train, cv=10, scoring='accuracy')
(cross_DTC_train.mean())*100

In [ ]:
cross_DTC_pred = cross_val_predict(model_DTC, X_train, y_train, cv=10)
cross_DTC_pred

In [ ]:
# evaluate the confusion matrix of multiple training rounds
confusion_matrix(y_train, cross_DTC_pred)

In [ ]:
precision_score(y_train, cross_DTC_pred), recall_score(y_train, cross_DTC_pred), f1_score(y_train, cross_DTC_pred)

In [ ]:
model_DTC_ = DecisionTreeClassifier(max_depth=7)
cross_DTC_pred = cross_val_predict(model_DTC_, X_train, y_train, cv=10, method='predict_proba')
cross_DTC_pred[:10]

In [ ]:
cross_DTC_score = cross_DTC_pred[:, 1] # labels of the +ve class only.`
prec_dtc, recall_dtc, thre_dtc = precision_recall_curve(y_train, cross_DTC_score)

In [ ]:
precision_recall_curve_plot(prec_dtc, recall_dtc, thre_dtc)
precision_recall_direct_plot(prec_dtc, recall_dtc)

In [ ]:
fpr_dtc, tpr_dtc, thre_dtc_roc = roc_curve(y_train, cross_DTC_score)
roc_curve_plot(fpr_dtc, tpr_dtc)
roc_auc_score(y_train, cross_DTC_score)

In [ ]:
# let's try the random forest classifier
from sklearn.ensemble import RandomForestClassifier

model_RFC = RandomForestClassifier()

In [ ]:
model_RFC.fit(X_train, y_train)

In [ ]:
RFC_train_pred = model_RFC.predict(X_train)
RFC_train_pred[:10]

In [ ]:
y_train.values[:10]

In [ ]:
# accuracy on multiple training processes
cross_RFC_train_score = cross_val_score(model_RFC, X_train, y_train, cv=10, scoring='accuracy')
(cross_RFC_train_score.mean())*100

In [ ]:
# confusion matrix for Random forest classifier
cross_RFC_pred = cross_val_predict(model_RFC, X_train, y_train, cv=5)
confusion_matrix(y_train, cross_RFC_pred)

In [ ]:
cross_RFC_pred

In [ ]:
precision_score(y_train, cross_RFC_pred), recall_score(y_train, cross_RFC_pred), f1_score(y_train, cross_RFC_pred)

In [ ]:
cross_RFC_score = cross_val_predict(model_RFC, X_train, y_train, cv=10, method='predict_proba')
cross_RFC_score # we got the probability for each customer to be within which class either: churned or didn't churn

In [ ]:
# let's evaluate the model performance with the precision/recall metrics
#  get the +ve class probability only as the score
cross_RFC_score = cross_RFC_score[:, 1]
prec_rfc, recall_rfc, thre_rfc = precision_recall_curve(y_train, cross_RFC_score)
precision_recall_curve_plot(prec_rfc, recall_rfc, thre_rfc)
precision_recall_direct_plot(prec_rfc, recall_rfc)

In [ ]:
fpr_rfc, tpr_rfc, thre_rfc_roc = roc_curve(y_train, cross_RFC_score)
roc_curve_plot(fpr_rfc, tpr_rfc)
roc_auc_score(y_train, cross_RFC_score)

We concluded that the **`random forest classifier`** is the most promising model to be used.
With **`AUC` = 92.37%**

Let's `fine-tune` our model

In [ ]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

param_dist = [
    {'n_estimators': randint(low=1, high=200), 'max_features':randint(low=1, high=17), 'max_depth': randint(low=1, high=50)}
]

model_RFC_randcv = RandomizedSearchCV(model_RFC, param_dist, cv=3, random_state=42, return_train_score=True, verbose=2)

In [ ]:
model_RFC_randcv.fit(X_train, y_train)

In [ ]:
model_RFC_randcv.best_estimator_, model_RFC_randcv.best_score_, model_RFC_randcv.

In [ ]:
cross_RFC_pred = cross_val_predict(model_RFC_randcv.best_estimator_, X_train, y_train, cv=10)

In [ ]:
confusion_matrix(y_train, cross_RFC_pred)

In [ ]:
cross_RFC_score = cross_val_predict(model_RFC_randcv.best_estimator_, X_train, y_train, cv=10, method='predict_proba')
cross_RFC_score = cross_RFC_score[:, 1] # the proba of the +ve class to be used as a score

In [ ]:
# let's evaluate the model performance with the precision/recall metrics
#  get the +ve class probability only as the score
prec_rfc, recall_rfc, thre_rfc = precision_recall_curve(y_train, cross_RFC_score)
precision_recall_curve_plot(prec_rfc, recall_rfc, thre_rfc)
precision_recall_direct_plot(prec_rfc, recall_rfc)

In [ ]:
fpr_rfc, tpr_rfc, thre_rfc_roc = roc_curve(y_train, cross_RFC_score)
roc_curve_plot(fpr_rfc, tpr_rfc)
roc_auc_score(y_train, cross_RFC_score)

In [ ]:
model_RFC_randcv.best_estimator_ # Done!

### Let's evaluate the model performance on the testing dataset


In [ ]:
final_test_data = full_pipe.fit_transform(test_df)

In [ ]:
final_test_data.shape

In [ ]:
final_test_data[-10:]

In [ ]:
# let's drop the target labels
X_test = np.c_[final_test_data[:, 0], final_test_data[:, 2:]]
y_test = test_df.Exited.copy()

In [ ]:
rfc_final_pred = model_RFC_randcv.best_estimator_.predict(X_test)
confusion_matrix(y_test, rfc_final_pred)

In [ ]:
precision_score(y_test, rfc_final_pred), recall_score(y_test, rfc_final_pred), f1_score(y_test, rfc_final_pred)

In [ ]:
rfc_final_score = model_RFC_randcv.best_estimator_.predict_proba(X_test)

In [ ]:
rfc_final_score = rfc_final_score[:, 1] # we take the +ve class probability as the score

In [ ]:
prec_rfc, recall_rfc, thre_rfc = precision_recall_curve(y_test, rfc_final_score)
precision_recall_curve_plot(prec_rfc, recall_rfc, thre_rfc)
precision_recall_direct_plot(prec_rfc, recall_rfc)

In [ ]:
fpr_rfc, tpr_rfc, thre_rfc_roc = roc_curve(y_test, rfc_final_score)
roc_curve_plot(fpr_rfc, tpr_rfc)
roc_auc_score(y_test, rfc_final_score)

In [ ]:
accuracy_score(y_test, rfc_final_pred)*100

### This is the end of our classification Journey  ;)

Final results on the testing dataset after fine-tuning: <br>
1. `Precision: 93.23%`<br>
2. `Recall:    86.06%`<br>
3. `accuracy:  95.8%`<br>

### Saving our final model

In [ ]:
import joblib
joblib.dump(model_RFC_randcv.best_estimator_, 'final_RFC_fine_tuned.pkl')

### Saving our final transformed testing dataset

In [ ]:
test_df_transformed = pd.DataFrame(final_test_data, columns=list_transformed_attrs)
test_df_transformed.head()

In [ ]:
test_df_transformed.to_csv('datasets/churnd/test_df_transformed.csv', index=False)